In [2]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""

from __future__ import absolute_import, division, print_function

import argparse
import glob
import logging
import os
import pickle
import random
import re
import shutil

import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from torch.utils.data.distributed import DistributedSampler
import json
# try:
#     from torch.utils.tensorboard import SummaryWriter
# except:
#     from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch
from torch.autograd import Variable
import copy
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm, trange
import multiprocessing
from model import Model
cpu_cont = multiprocessing.cpu_count()
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          BertConfig, BertForMaskedLM, BertTokenizer,
                          GPT2Config, GPT2LMHeadModel, GPT2Tokenizer,
                          OpenAIGPTConfig, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer,
                          RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer,
                          DistilBertConfig, DistilBertForMaskedLM, DistilBertTokenizer)

logger = logging.getLogger(__name__)

MODEL_CLASSES = {
    'gpt2': (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),
    'openai-gpt': (OpenAIGPTConfig, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
    'bert': (BertConfig, BertForMaskedLM, BertTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer),
    'distilbert': (DistilBertConfig, DistilBertForMaskedLM, DistilBertTokenizer)
}

In [64]:
    
class Model(nn.Module):   
    def __init__(self, encoder,config,tokenizer):
        super(Model, self).__init__()
        self.encoder = encoder
        self.config=config
        self.tokenizer=tokenizer
        # self.args=args
        # self.softmax = torch.softmax(dim=-1)
        self.criterion = nn.CrossEntropyLoss()
    
        
    def forward(self, input_ids, inputs_embeds=None,labels=None): 
        outputs=self.encoder(inputs_embeds=inputs_embeds, attention_mask=input_ids.ne(1))
        logits=outputs[0]
        prob=torch.softmax(logits, dim=-1)
        if labels is not None:
            print(prob.shape, labels.shape)
            return self.criterion(prob, labels), prob, outputs[1], outputs[2]
        else:
            return prob, outputs[1], outputs[2]


class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 input_tokens,
                 input_ids,
                 idx,
                 label,

    ):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.idx=str(idx)
        self.label=label

def convert_examples_to_features(js,tokenizer,args):
    #source
    code=' '.join(js['func'].split())
    code_tokens=tokenizer.tokenize(code)[:args.block_size-2]
    source_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
    source_ids =  tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = args.block_size - len(source_ids)
    source_ids+=[tokenizer.pad_token_id]*padding_length
    return InputFeatures(source_tokens,source_ids,js[args.idx_key],js['target'])

class TextDataset(Dataset):
    def __init__(self, tokenizer, args, file_path=None):
        self.examples = []
        with open(file_path) as f:
            for line in f:
                js=json.loads(line.strip())
                self.examples.append(convert_examples_to_features(js,tokenizer,args))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        labels = np.zeros((2,))
        labels[self.examples[i].label] = 1
        return torch.tensor(self.examples[i].input_ids), torch.tensor(self.examples[i].label)


class CodebertModel:
    """Simple sentiment analysis model."""

    LABELS = [0, 1]  # negative, positive
    compute_grads: bool = True  # if True, compute and return gradients.
    config_class, model_class, tokenizer_class = RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer

    def __init__(self):
        self.model_name_or_path = 'microsoft/codebert-base'
        self.model_config = self.config_class.from_pretrained(self.model_name_or_path,
                num_labels=2,
                output_hidden_states=True,
                output_attentions=True)
        self.tokenizer = self.tokenizer_class.from_pretrained('microsoft/codebert-base')

        model = self.model_class.from_pretrained(self.model_name_or_path,
                                                from_tf=bool('.ckpt' in self.model_name_or_path),
                                                 config=self.model_config)
        self.model = Model(model, self.model_config,  self.tokenizer)
        self.output_dir="./saved_models"

    def activate_evaluation(self):
        self.model.eval()

    def load_model(self, args):
        checkpoint_prefix = 'checkpoint-best-acc/model.bin'
        output_dir = os.path.join(args.output_dir, '{}'.format(checkpoint_prefix))
        self.model.load_state_dict(torch.load(output_dir, map_location=args.device))
        self.model.to(args.device)


    def test(self, args):
        model = self.model
        tokenizer = self.tokenizer
        # Loop to handle MNLI double evaluation (matched, mis-matched)
        eval_dataset = TextDataset(tokenizer, args,args.test_data_file)


        # args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
        # Note that DistributedSampler samples randomly
        eval_sampler = SequentialSampler(eval_dataset) if args.local_rank == -1 else DistributedSampler(eval_dataset)
        eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

        # multi-gpu evaluate
        # if args.n_gpu > 1:
        #     model = torch.nn.DataParallel(model)

        # Eval!
        logger.info("***** Running Test *****")
        logger.info("  Num examples = %d", len(eval_dataset))
        logger.info("  Batch size = %d", args.eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0
        model.eval()
        logits=[]   
        labels=[]
        for batch in tqdm(eval_dataloader,total=len(eval_dataloader)):
            inputs = batch[0].to(args.device)        
            label=batch[1].to(args.device) 
            with torch.no_grad():
                logit = model(inputs)
                logits.append(logit.cpu().numpy())
                labels.append(label.cpu().numpy())

        logits=np.concatenate(logits,0)
        labels=np.concatenate(labels,0)

        acts = np.array([0 if (sample[0] > sample[1]) else 1 for sample in labels])
        preds = np.array([0 if (sample[0] > sample[1]) else 1 for sample in logits])
        
        eval_acc=np.mean(acts==preds)
        print(eval_acc)
    

    def explain(self, input_ids, labels, steps=20, start_layer=4):
        self.activate_evaluation()
        input = self.model.encoder.roberta.embeddings(input_ids)
        b = input.shape[0]

        output, _, attention = self.model(input_ids, inputs_embeds=input)

        b, h, s, _ = attention[-1].shape
        num_blocks = len(attention)

        states = attention[-1].mean(1)[:, 0, :].reshape(b, 1, s)
        for i in range(start_layer, num_blocks - 1)[::-1]:
            attn = attention[i].mean(1)
            states_ = states
            states = states.bmm(attn)
            states += states_

        total_gradients = torch.zeros(b, h, s, s).cpu()
        for alpha in np.linspace(0, 1, steps):        
            # forward propagation
            data_scaled = input * alpha
            # backward propagation
            output, _, attention = self.model(input_ids, data_scaled)
            one_hot = np.zeros((b, 2), dtype=np.float32)
            one_hot[np.arange(b), labels] = 1
            one_hot = torch.from_numpy(one_hot).requires_grad_(True)
            one_hot = torch.sum(one_hot.cpu() * output)

            self.model.zero_grad()
            attention[-1].retain_grad()
            one_hot.backward(retain_graph=True)

            # cal grad
            gradients = attention[-1].grad
            total_gradients += gradients

        W_state = (total_gradients / steps).clamp(min=0).mean(1)[:, 0, :].reshape(b, 1, s)
        states = states * W_state
        return states[:, 0, :]

    def explanation(self, args):
        self.weights = self.model.encoder.get_input_embeddings()
        model = self.model
        tokenizer = self.tokenizer
        # Loop to handle MNLI double evaluation (matched, mis-matched)
        eval_dataset = TextDataset(tokenizer, args, args.test_data_file)

        eval_sampler = SequentialSampler(eval_dataset) if args.local_rank == -1 else DistributedSampler(eval_dataset)
        eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

        for id, batch in enumerate(eval_dataloader):
            input_ids, labels = batch
            scores = self.explain(input_ids, labels)
            print(scores)


class Arg:
    def __init__(self):
        self.train_data_file = '../dataset/train.jsonl'
        self.device = torch.device('cpu')
        self.epoch = 5
        self.train_batch_size = 16
        self.adam_epsilon = 1e-8
        self.learning_rate = 2e-5
        self.max_grad_norm=1.0
        self.weight_decay = 0.0
        self.gradient_accumulation_steps = 1
        self.local_rank = -1
        self.output_dir = '/Users/mahbubcseju/Desktop/projects/TransformerExplainability/saved_models/func_jsonal/64'
        self.eval_data_file = '../dataset/valid.jsonl'
        self.eval_batch_size = 1
        self.evaluate_during_training = True
        self.test_data_file = '/Users/mahbubcseju/Desktop/projects/TransformerExplainability/data/func_jsonal/test.jsonl'
        self.idx_key='id'
        self.block_size=400


def main():
    args = Arg()

    model = CodebertModel()
    # model.train(args)
    model.load_model(args)
    model.explanation(args)
    # model.test(args)
    print("Sucess")

if __name__ == "__main__":
    main()


Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be 

Second  torch.Size([1, 400, 400])
Second  torch.Size([1, 400, 400])
Second  torch.Size([1, 400, 400])
Second  torch.Size([1, 400, 400])
Second  torch.Size([1, 400, 400])
Second  torch.Size([1, 400, 400])
Second  torch.Size([1, 400, 400])
torch.Size([1, 1, 400])
States:  torch.Size([1, 1, 400])
Start total gradients  torch.Size([1, 12, 400, 400])
step gradients size :  torch.Size([1, 12, 400, 400])
Step total gradients  torch.Size([1, 12, 400, 400])
step gradients size :  torch.Size([1, 12, 400, 400])
Step total gradients  torch.Size([1, 12, 400, 400])
step gradients size :  torch.Size([1, 12, 400, 400])
Step total gradients  torch.Size([1, 12, 400, 400])
step gradients size :  torch.Size([1, 12, 400, 400])
Step total gradients  torch.Size([1, 12, 400, 400])
step gradients size :  torch.Size([1, 12, 400, 400])
Step total gradients  torch.Size([1, 12, 400, 400])
step gradients size :  torch.Size([1, 12, 400, 400])
Step total gradients  torch.Size([1, 12, 400, 400])
step gradients size : 